# Generate multiple [character.ai](https://beta.character.ai/) character definitions

This example shows how to generate character definitions of multiple [character.ai](https://beta.character.ai/) characters from a corpus. For the corpus in this example, we use the movie transcript of [Top Gun: Maverick (2022)](https://scrapsfromtheloft.com/movies/top-gun-maverick-transcript/).

To generate your own character definitions:
1. Put the corpus into a single a `.txt` file inside the `data/` directory.
2. Assign the name of the `.txt` file to the `CORPUS` constant below.
3. Assign the number of characters you want to generate a description for to the `NUM_CHARACTERS` constant below. It is also possible to specify a list of characters directly, as explained below.
4. Run this notebook.

In [1]:
CORPUS = 'data/top_gun_maverick.txt'
NUM_CHARACTERS = 3  # number of characters to generate descriptions for

In [2]:
from dataclasses import asdict
import json
import os

from src.character import get_character_definition
from src.corpus import get_characters, get_rolling_summaries, load_docs

In [3]:
# create directories to cache results and intermediate outputs
OUTPUT_ROOT = "output"
corpus_name = os.path.splitext(os.path.basename(CORPUS))[0]
output_dir = f"{OUTPUT_ROOT}/{corpus_name}"
os.makedirs(output_dir, exist_ok=True)
summaries_dir = f"{output_dir}/summaries"
character_definitions_dir = f"{output_dir}/character_definitions"
os.makedirs(character_definitions_dir, exist_ok=True)

## Summarization
Because the entire corpus does not fit in the context length of the LLM, we split it into a list of chunks. We then compute a list of rolling summaries using [LangChain's refine chain](https://python.langchain.com/en/latest/modules/chains/index_examples/summarize.html#the-refine-chain). We first summarize the first chunk. Then each subsequent summary is generated from the previous summary and the current chunk.

Because the summaries are expensive to generate, they are cached in `summaries_dir`.

In [ ]:
# split corpus into a set of chunks
docs = load_docs(
    corpus_path=CORPUS,
    chunk_size=2048,  # number of tokens per chunk
    chunk_overlap=64,  # number of tokens of overlap between chunks
)

# generate rolling summaries
rolling_summaries = get_rolling_summaries(docs=docs, cache_dir=summaries_dir)

## Generate a list of characters
We can automatically generate a list of the main characters in the corpus, as shown below. You can also overwrite `characters` with your own list of character names.

In [5]:
# generate list of characters
characters = get_characters(
    rolling_summaries=rolling_summaries,
    num_characters=NUM_CHARACTERS,
    cache_dir=output_dir,
)
print(characters)

['Pete "Maverick" Mitchell', 'Bradley "Rooster" Bradshaw', 'Admiral Cain']


## Generate [character.ai](https://beta.character.ai/) character definitions
Based on the corpus, we can now generate the elements - name, short description (50 characters), long description (500 characters), and custom greeting - that are required to [create a character.ai character](https://beta.character.ai/editing). These character definitions are cached in `character_definitions_dir`. You can then [place these characters can in a room](https://beta.character.ai/room/create?) and watch them converse!

In [9]:
character_definitions = []
for character in characters:
    character_definition = get_character_definition(
        name=character,
        rolling_summaries=rolling_summaries,
        cache_dir=character_definitions_dir,
    )
    character_definitions.append(character_definition)

In [10]:
for character_definition in character_definitions:
    print(json.dumps(asdict(character_definition), indent=4))

{
    "name": "Pete \"Maverick\" Mitchell",
    "short_description": "You're a talented Navy pilot and mentor.",
    "long_description": "You're a former Navy aviator, called back to train TOP GUN graduates for a dangerous mission to destroy an unsanctioned uranium plant. Struggling with loyalty to your late friend Goose and responsibility as a trainer, especially training Goose's son. You reconnect with an old flame and face past rivals. Grounded and replaced, you prove your flying skills, become team leader, and come to terms with Goose's loss.",
    "greeting": "Hey there, I'm Maverick. Ready to take to the skies?"
}
{
    "name": "Bradley \"Rooster\" Bradshaw",
    "short_description": "You're Goose's son, skilled pilot, determined.",
    "long_description": "You are Goose's son, who died tragically. Eager to follow in your father's footsteps, you join the TOP GUN program and become one of six elite graduates trained by Maverick. Struggling with the weight of your father's legacy a